In [1]:
# read csv data file
import csv
f = open("module2_quiz_P1.csv")
csvfile = csv.DictReader(f)

headers = csvfile.fieldnames

table = []
for row in csvfile:
    table.append(row)
    
f.close()

In [2]:
# define set of Segments
Segments = []
for row in table:
    if row['Segment'] != 'Cost per ad':
        Segments.append(row['Segment'])
        
# define set of Outlets
Outlets = headers[:]
Outlets.remove('Segment')
Outlets.remove('Minimum Impressions')

In [3]:
# define parameters
Cost = {}
for o in Outlets:
    Cost[o] = float(table[-1][o])

Demand = {}
for row in table:
    if row['Segment'] != 'Cost per ad':
        Demand[row['Segment']] = float(row['Minimum Impressions'])
        
Impressions = {}
for row in table:
    if row['Segment'] != 'Cost per ad':
        for o in Outlets:
            Impressions[(row['Segment'],o)] = float(row[o])

In [4]:
print(Impressions)

{('WWC', 'R'): 500.0, ('WWC', 'TV1'): 3000.0, ('WWC', 'TV2'): 2000.0, ('WWC', 'TV3'): 4000.0, ('WWC', 'W'): 25.0, ('WWC', 'M'): 50.0, ('TTL', 'R'): 3.0, ('TTL', 'TV1'): 150.0, ('TTL', 'TV2'): 250.0, ('TTL', 'TV3'): 300.0, ('TTL', 'W'): 1700.0, ('TTL', 'M'): 2000.0, ('NEN', 'R'): 1500.0, ('NEN', 'TV1'): 2500.0, ('NEN', 'TV2'): 4000.0, ('NEN', 'TV3'): 2000.0, ('NEN', 'W'): 150.0, ('NEN', 'M'): 0.0, ('ILC', 'R'): 0.0, ('ILC', 'TV1'): 500.0, ('ILC', 'TV2'): 1000.0, ('ILC', 'TV3'): 1500.0, ('ILC', 'W'): 100.0, ('ILC', 'M'): 0.0}


In [5]:
from docplex.mp.model import Model
mdl = Model()

In [6]:
# Define decision variables
NumberofAds = mdl.continuous_var_dict(Outlets, lb=0, name="NumberofAds")

In [7]:
# Define objective function
total_cost = mdl.sum(Cost[o] * NumberofAds[o] for o in Outlets)
mdl.minimize(total_cost)

In [8]:
# Define constraints
for s in Segments:
    mdl.add_constraint(mdl.sum(Impressions[(s, o)] * NumberofAds[o] for o in Outlets) >= Demand[s])

In [9]:
# solve
mdl.solve()
mdl.get_solve_details()

docplex.mp.SolveDetails(time=0,status='optimal')

In [10]:
mdl.print_solution()


objective: 11348.667
status: OPTIMAL_SOLUTION(2)
  NumberofAds_TV3=34.667
  NumberofAds_M=29.800


In [11]:
# Add new constraint for web and mobile advertising spending
mdl.add_constraint(100 * NumberofAds['W'] + 90 * NumberofAds['M'] <= 0.15 * total_cost)


docplex.mp.LinearConstraint[](100NumberofAds_W+90NumberofAds_M,LE,11.250NumberofAds_R+41.250NumberofAds_TV1+45NumberofAds_TV2+37.500NumberofAds_TV3+15NumberofAds_W+13.500NumberofAds_M)

In [12]:
mdl.solve()
mdl.get_solve_details()

docplex.mp.SolveDetails(time=0,status='optimal')

In [13]:
mdl.print_solution()

objective: 16079.632
status: OPTIMAL_SOLUTION(2)
  NumberofAds_TV3=54.671
  NumberofAds_M=26.799
